# Convert Masks to YOLO Annotations
### Dataset Format for YOLO
YOLO requires annotations in .txt files (one per image) with bounding boxes or segmentation masks. For object detection (bounding boxes), use this format:
```bash
0 0.5 0.5 0.2 0.3  # class_id, x_center, y_center, width, height
1 0.7 0.3 0.1 0.1
```
If your masks are binary images, use OpenCV to extract bounding boxes:

In [5]:
import cv2
import os
import numpy as np
from pathlib import Path

def mask_to_yolo(mask_path, output_txt_path, class_id, mode='segmentation'):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Could not read mask {mask_path}")
        return
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    with open(output_txt_path, 'w') as f:
        for contour in contours:
            if mode == 'segmentation':
                contour = contour.squeeze()
                if contour.ndim == 1:
                    continue  # Skip invalid contours
                contour_norm = contour / np.array([mask.shape[1], mask.shape[0]])
                f.write(f"{class_id} " + " ".join([f"{x:.5f} {y:.5f}" for x,y in contour_norm]) + "\n")
            elif mode == 'detection':
                x, y, w, h = cv2.boundingRect(contour)
                x_center = (x + w/2) / mask.shape[1]
                y_center = (y + h/2) / mask.shape[0]
                width = w / mask.shape[1]
                height = h / mask.shape[0]
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {width:.5f} {height:.5f}\n")

# Process all splits (match your folder names: train, valid, test)
for split in ['train', 'valid', 'test']:  # Fixed 'val' -> 'valid'
    split_dir = Path(f'/home/user/Documents/LRS/mask/{split}')
    image_dir = split_dir / 'images'
    label_dir = split_dir / 'labels'
    
    # Create both images and labels directories
    image_dir.mkdir(parents=True, exist_ok=True)  # Fix: Create images directory
    label_dir.mkdir(parents=True, exist_ok=True)
    
    # Process all masks in the split directory
    for mask_path in split_dir.glob('*_mask.png'):
        # Get corresponding image (e.g., xyz_mask.png → xyz.jpg)
        image_path = mask_path.with_name(mask_path.name.replace('_mask.png', '.jpg'))
        
        # Move image to images subdirectory
        try:
            image_path.rename(image_dir / image_path.name)
        except FileNotFoundError:
            print(f"Error: Image {image_path} not found. Skipping...")
            continue
        
        # Generate YOLO annotation file
        output_txt = label_dir / f"{mask_path.stem.replace('_mask', '')}.txt"
        mask_to_yolo(mask_path, output_txt, class_id=0, mode='segmentation')  # Adjust mode as needed

# Segmentation (YOLOv8)

Install YOLOv8:

In [ ]:
!pip3 install ultralytics

## Training Code

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # Segmentation model
model.train(data='/home/user/Documents/LRS/data.yaml', project="/home/user/Documents/LRS/Models", epochs=50, imgsz=640, batch=16)

In [ ]:
import cv2

# Load the trained model
model = YOLO('/home/user/Documents/RSI/runs/segment/train2/weights/best.pt')

# Predict on a single image
image_path = "/home/user/Documents/LRS/mask/test/images/tile_12_jpg.rf.b0d051c082e97538bdd45a416fa6d93b.jpg"
results = model.predict(image_path, imgsz=640, conf=0.5)

# Visualize results
for result in results:
    plotted = result.plot()  # Draw masks/boxes on the image
    cv2.imshow("Prediction", plotted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()